In [1]:
# resources
import cloudpickle as cp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import PyKEP as pk
import itertools
%matplotlib

Using matplotlib backend: Qt5Agg


In [ ]:
# get data
data = cp.load(open("../p/random_moc_walks.p", "rb"))

# number of attempted trajectories
na = len(data)
# number of infeasible trajectories
ni = len([traj for traj in data if traj is None])
# number of unconverged trajectories
nu = len([traj for traj in data if traj is not None and traj.leg.alpha != 1])
# number of succesfull trajectories
ns = len([traj for traj in data if traj is not None and traj.leg.alpha == 1])

# percent succesfull out of attempted
psa = ns/na
# percent succesfull out of feasible
psf = ns/nu

print(psa*100, psf*10)

In [ ]:
# get trajectory data
data = [traj.leg.get_states() for traj in data if traj is not None and traj.leg.alpha == 1]

In [ ]:
data = np.vstack(data)
xudata = np.hstack((data[:, 1:8], data[:, 16:20]))
np.save("../npy/moc_data.npy", data)
np.save("../npy/xu_moc_data.npy", xudata)

In [ ]:
xudata = np.load("../npy/xu_moc_data.npy")

In [ ]:
pk.ic2par(xudata[0, 0:3], xudata[0, 3:6], pk.MU_SUN)

In [ ]:
keps = np.array([pk.ic2par(s[0:3], s[3:6], pk.MU_SUN) for s in xudata])

In [ ]:
np.save("../npy/kep_moc_data.npy", keps)

# Keplerian Joint Plots

In [ ]:
keps = np.load("../npy/kep_moc_data.npy")
df = pd.DataFrame(keps, columns=["a [AU]", "e", "i [rad]", "$\Omega$ [rad]", "$\omega$ [rad]", "E [rad]"])

In [ ]:
g = sns.pairplot(df, palette=sns.color_palette("Blues_d"))

In [ ]:
g.savefig("moc_kep_scatter.png")

# QC and MOC Data

In [ ]:
qcw = cp.load(open("../p/random__qc_walks.p", "rb"))
mcw = cp.load(open("../p/random_moc_walks.p", "rb"))

In [ ]:
xu_qc = np.empty(shape=(0, 11))
for prob in qcw:
    data = prob.leg.get_states()
    state = data[:, 1:8]
    control = data[:, 16:20]
    data = np.hstack((state, control))
    xu_qc = np.vstack((xu_qc, data))
np.save("xu_qc.npy", xu_qc)

xu_moc = np.empty(shape=(0, 11))
for prob in mcw:
    if prob is not None and prob.leg.alpha == 1:
        data = prob.leg.get_states()
        state = data[:, 1:8]
        control = data[:, 16:20]
        data = np.hstack((state, control))
        xu_moc = np.vstack((xu_moc, data))
np.save("xu_moc.npy", xu_moc)

In [2]:
xu_qc = np.load("../npy/xu_qc.npy")
xu_moc = np.load("../npy/xu_moc.npy")

In [3]:
# turn Cartesian data into spherical
datas = list()
for data in [xu_qc, xu_moc]:
    
    # Cartesian direction
    ux    = data[:, 8]
    uy    = data[:, 9]
    uz    = data[:, 10]
    
    # radius [0, inf]
    r = np.sqrt(ux**2 + uy**2 + uz**2)
    
    # inclination [-pi, pi]
    theta = np.arctan2(uy, ux)
    
    # azimuth [0, pi]
    phi = np.arccos(uz/r)
    
    # number of data points
    ndata = len(data)
    
    # reshape
    theta = theta.reshape((ndata, 1))
    phi   = phi.reshape((ndata, 1))
    
    # add the data
    datas.append(np.hstack((data[:, 0:8], theta, phi)))
    
xu_qc, xu_moc = datas

In [ ]:
np.save("../npy/xu_qc_sph.npy", xu_qc)
np.save("../npy/xu_moc_sph.npy", xu_moc)

In [4]:
print(min(xu_qc[:, 8]), max(xu_qc[:, 8]))
print(min(xu_qc[:, 9]), max(xu_qc[:, 9]))

print(min(xu_qc[:, 8]), max(xu_qc[:, 8]))
print(min(xu_qc[:, 9]), max(xu_qc[:, 9]))

-3.14138566523 3.14157726342
0.000490158274372 3.13979638322


In [5]:
xu_moc[:, 7:]

array([[ 1.        , -2.00500559,  1.26627199],
       [ 1.        , -1.99243581,  1.26802578],
       [ 1.        , -1.917374  ,  1.27961714],
       ..., 
       [ 1.        ,  1.86157372,  1.3612974 ],
       [ 1.        ,  1.86753933,  1.35279692],
       [ 1.        ,  1.88039425,  1.33462444]])

In [6]:
xu_qc[:, 7:]

array([[ 0.68156988, -2.04851941,  1.1985929 ],
       [ 0.68017243, -2.03644936,  1.1996701 ],
       [ 0.67120374, -1.96443699,  1.20731823],
       ..., 
       [ 0.45197212,  2.85997413,  1.2510391 ],
       [ 0.45302967,  2.97482489,  1.26462476],
       [ 0.45342857,  3.02489562,  1.27184467]])

In [ ]:
x = np.cos(xu_moc[:, 8])*np.sin(xu_moc[:, 9])
y = np.sin(xu_moc[:, 8])*np.sin(xu_moc[:, 9])
z = np.cos(xu_moc[:, 8])


In [ ]:
x == xu_moc[:, 7]

In [ ]:
y